In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
# import data_helpers
from tensorflow.contrib import learn
import csv
from pathlib import Path 
import gensim
import pickle
import pandas as pd
from sklearn import preprocessing
import re

In [2]:
data_dir_path = os.path.join(Path(os.getcwd()).parent.parent, 'deeptech-ai-data')

In [3]:
# load model
model_path = os.path.join(data_dir_path, 'german.model')
trained_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
vocab = list(trained_model.vocab)

In [8]:
def get_df_companies_labeling(file):
    with open(file, 'rb') as pickle_file:
        companies_labeling = pickle.load(pickle_file)
    
    df = pd.DataFrame(companies_labeling)
    
    categories = list(set(df['section_name']))
    
    le = preprocessing.LabelEncoder()
    le.fit(df['section'])
    df['y'] = le.transform(df['section'])
    
    return df

In [11]:
companies_labeling_with_cat_num = os.path.join(data_dir_path, 'companies_enchanced_labeling_cat.pickle')
df = get_df_companies_labeling(companies_labeling_with_cat_num)

In [14]:
def metadata_df_from_file(file):
    with open(file, 'rb') as pickle_file:
        metadata_df = pickle.load(pickle_file)
        
    return metadata_df

In [16]:
metadata_df_file = os.path.join(data_dir_path, 'metadata_df.pickle')
metadata_df = metadata_df_from_file(metadata_df_file)

In [17]:
metadata_df

,url,section,code,1,abstract,description,keywords
0,www.hoberg-driesch.de,L,68.31,(Qualitätsstahlrohre von Hoberg & Driesch. Von...,Qualitätsstahlrohre von Hoberg & Driesch. Von ...,Hoberg & Driesch GmbH & Co. KG ist eines der f...,"[Röhrengrosshandel, Starkwandrohre, Normalwand..."
1,www.hgdf.de,K,64.20,"(, , , False)",,,[]
2,www.gasag.de,D,35.30,"(, Bei der GASAG finden Sie die richtigen Stro...",,Bei der GASAG finden Sie die richtigen Strom- ...,"[Strom, Erdgas, Wärme, Privatkunden, Stromvert..."
3,www.ergodirekt.de,K,65.11,"(, ERGO Direkt ist meistgewählter Direktversic...",,ERGO Direkt ist meistgewählter Direktversicher...,[]
4,www.wahler.de,M,70.10,"(, , , False)",,,[]
5,www.msa-auer.de,S,95.11,"(, MSA is a global manufacturer of safety prod...",,MSA is a global manufacturer of safety product...,"[MSA, MSA safety, MSAsafety, safety products, ..."
6,www.nuernberger.de,M,70.10,"(, Die NÜRNBERGER Versicherung - Solide. Finan...",,Die NÜRNBERGER Versicherung - Solide. Finanzst...,"[NÜRNBERGER, Versicherung]"
7,www.ds-cc.de,M,73.11,"(, Wir sind eines der führenden Outsourcing-Un...",,Wir sind eines der führenden Outsourcing-Unter...,"[Customer Care Services, Kundenservices, Kunde..."
8,www.mlk-berlin.de,Q,86.10,"(, Martin-Luther-Krankenhaus - Seit 1931 in Be...",,Martin-Luther-Krankenhaus - Seit 1931 in Berli...,[]
9,www.goldhofer.de,C,29.10,"(, , Goldhofer Unternehmensgruppe, Goldhofer C...",,,"[Goldhofer Unternehmensgruppe, Goldhofer Compa..."


In [44]:
row = metadata_df[metadata_df['url'] == 'www.hoberg-driesch.de']
y = row.iloc[0]['keywords']

y

['Röhrengrosshandel',
 'Starkwandrohre',
 'Normalwandrohre',
 'Gewinderohre',
 'Kesselrohre',
 'Hohlprofile',
 'Präzisions\xadstahl\xadrohre',
 'HPZ-Rohre',
 'Wälzlager\xadstahl\xadrohre',
 'Feinkornbaustahlrohre',
 'Maschinenbaustahlrohre']